<a href="https://colab.research.google.com/github/young-hun-jo/KaggleStruggle/blob/master/jeju_credit/main_modeling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!ls '/gdrive/My Drive/dacon_카드사용액 예측/data'

201901-202003.csv  encoded_data.csv  submission.csv
202004.csv	   jeju_airport.csv


In [3]:
import pandas as pd
data = pd.read_csv('/gdrive/My Drive/dacon_카드사용액 예측/data/201901-202003.csv')
data.shape

(24697792, 12)

In [4]:
data.fillna('세종', inplace=True)

In [5]:
data2 = pd.read_csv('/gdrive/My Drive/dacon_카드사용액 예측/data/202004.csv')
data2.shape

(1350322, 12)

In [6]:
data = pd.concat([data, data2], axis=0)
data.shape

(26048114, 12)

In [7]:
def grap_year(data): # year 
    data = str(data)
    return int(data[:4])

def grap_month(data): # month 
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [8]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']
del data['HOM_CCG_NM']
del data['CSTMR_CNT']
del data['CNT']

In [9]:
sex_encoding = pd.get_dummies(data=data['SEX_CTGO_CD'], prefix='SEX')
data = pd.concat([data, sex_encoding], axis=1)
data.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,year,month,SEX_1,SEX_2
0,강원,건강보조식품 소매업,강원,20s,1,1,311200,2019,1,1,0
1,강원,건강보조식품 소매업,강원,30s,1,2,1374500,2019,1,1,0
2,강원,건강보조식품 소매업,강원,30s,2,2,818700,2019,1,0,1
3,강원,건강보조식품 소매업,강원,40s,1,3,1717000,2019,1,1,0
4,강원,건강보조식품 소매업,강원,40s,1,4,1047300,2019,1,1,0


In [10]:
del data['SEX_CTGO_CD']
data.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,FLC,AMT,year,month,SEX_1,SEX_2
0,강원,건강보조식품 소매업,강원,20s,1,311200,2019,1,1,0
1,강원,건강보조식품 소매업,강원,30s,2,1374500,2019,1,1,0
2,강원,건강보조식품 소매업,강원,30s,2,818700,2019,1,0,1
3,강원,건강보조식품 소매업,강원,40s,3,1717000,2019,1,1,0
4,강원,건강보조식품 소매업,강원,40s,4,1047300,2019,1,1,0


In [11]:
data = data[['year','month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_1','SEX_2','FLC','AMT']] # local_nlocal
data.head()

,year,month,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_1,SEX_2,FLC,AMT
0,2019,1,강원,건강보조식품 소매업,강원,20s,1,0,1,311200
1,2019,1,강원,건강보조식품 소매업,강원,30s,1,0,2,1374500
2,2019,1,강원,건강보조식품 소매업,강원,30s,0,1,2,818700
3,2019,1,강원,건강보조식품 소매업,강원,40s,1,0,3,1717000
4,2019,1,강원,건강보조식품 소매업,강원,40s,1,0,4,1047300


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from itertools import product
import numpy as np
import pandas as pd

## encoding def
## 인코딩 완료된 df형태로 반환
def encoder_func(df):
  dtypes = df.dtypes
  encoders = {}
  for column in df.columns:
    if str(dtypes[column]) == 'object':
      encoder = LabelEncoder()
      encoder.fit(df[column])
      encoders[column] = encoder

  for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

  return df,encoders


# 지역별 모델링
def modeling(full_data): # full_data는 전체 데이터의미. (data)

  res_model_dict = {} # 각 지역별 모델 저장할 딕셔너리
  res_forecasting = pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']) # res_forecating에 각 지역별 예측결과를 누적하여 저장.

  city_list = full_data['CARD_SIDO_NM'].unique() # 전체 지역 리스트
  
  for city in city_list : # 각 지역별로 반복함.
    print(city) 
    sampling = full_data.query("CARD_SIDO_NM == @city") # 전체 데이터에서 한 지역씩 sampling 변수에 담음
    Nonexistent_std_clss = list( set(full_data['STD_CLSS_NM'].unique()) - set(sampling['STD_CLSS_NM'].unique())) # 각 지역별로 집계값이 0인 업종들
    sampling,encoders = encoder_func(sampling) # 위에서 정의한 인코딩함수에 sampling을 넣어 인코딩 된 데이터와 인코더 객체를 반환받음

    train_num = sampling.copy()
    train_features = train_num.drop(['CARD_SIDO_NM','AMT'],axis=1) # features # 각 지역별로 진행하기 때문에 CARD_SIDO_NM은 제외함.
    train_target = np.log1p(train_num['AMT']) # target

    model = RandomForestRegressor(n_jobs=-1,random_state=42) # model 선언 
    model.fit(train_features, train_target) # 학습

    res_model_dict['model_'+city] = model # 학습 된 모델을 res_model 딕셔너리에 저장

    ## 템플릿 생성 부분
    ## 각 지역별로 진행하기 때문에 CARD_SIDO_NM은 제외함.
    year_temp = [2020]
    month_temp = [7]
    STD_CLSS_NM_temp = sampling['STD_CLSS_NM'].unique()
    HOM_SIDO_NM_temp = sampling['HOM_SIDO_NM'].unique()
    AGE_temp = sampling['AGE'].unique()
    SEX_1_temp = sampling['SEX_1'].unique()
    SEX_2_temp = sampling['SEX_2'].unique()
    FLC_temp = sampling['FLC'].unique()
    # local_nlocal_temp = sampling['local_nlocal'].unique() # 이외에 변수 추가시 같은 방법으로 추가해주면 됨
    comb_list = [year_temp, month_temp, STD_CLSS_NM_temp, HOM_SIDO_NM_temp, AGE_temp, SEX_1_temp, SEX_2_temp, FLC_temp] # local_nlocal_temp
    forecasting_temp = np.array(list(product(*comb_list)))
    forecasting_temp = pd.DataFrame(data=forecasting_temp, columns=train_features.columns) # 예측 템플릿 완성

    pred = model.predict(forecasting_temp) # 예측
    pred = np.expm1(pred)
    forecasting_temp['AMT'] = np.round(pred, 0) 
    forecasting_temp['REG_YYMM'] = forecasting_temp['year']*100 + forecasting_temp['month']
    forecasting_temp = forecasting_temp[['REG_YYMM','STD_CLSS_NM', 'AMT']]
    forecasting_temp.insert(1,'CARD_SIDO_NM',city) # 첫번째 열에 "CARD_SIDO_NM"이라는 열을 추가함 값은 해당 지역이름임
    forecasting_temp = forecasting_temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False) # 지역별,업종별 월간 사용금액 합계
    forecasting_temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(forecasting_temp['STD_CLSS_NM']) # STD_CLSS_NM inverse encoding
    
    ## 각 지역별로 집계값이 0인 업종들이 있음
    ## submission에서는 이 업종들도 포함해서 제출해야 하기 때문에 0으로 때려넣을것임
    for clss in Nonexistent_std_clss:
      mm = [202007]
      tmp = pd.DataFrame([[ mm,city,clss,0 ]],columns=forecasting_temp.columns)
      forecasting_temp = pd.concat([forecasting_temp,tmp]) # forecasting_temp에 concat
    
    res_forecasting = pd.concat([res_forecasting,forecasting_temp]) # res_forecasting에 지역별 예측완료 된 값들을 누적하여 저장
    
  return res_model_dict,res_forecasting # 지역별 모델 딕셔너리, 최종 예측값 반환

res_model_dict,res_forecasting = modeling(data) # 실행


강원


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


경기


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


경남


...

경북


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


광주


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


대구
